In [1]:
from feat_eng_prod import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

2019-12-01-14-55


**Observations with Labels**
---
Using the icd-hcc crosswalk, the icd diagnosis data for each admission and the admissions table, create a labeled dataset with the target variable.

In [2]:
icdxw = read_crosswalk()
diagnoses_icd = read_prod_data('diagnoses_icd')
admissions = read_prod_data('admissions')
df = add_aki_hcc_label(diagnoses_icd, icdxw, admissions)
hcc_data = create_hcc_labeled_dataset(diagnoses_icd, icdxw)

printer('labeled dataset')
df.shape
df.sample(5)
df.hcc_cd_135.value_counts(normalize=True)
del diagnoses_icd
del icdxw

diagnoses_icd (651047, 5)
admissions (58976, 19)
labeled dataset
-------------------


(58976, 5)

,hadm_id,subject_id,hcc_cd_135,admittime,dischtime
37967,164446,82187,0,2150-05-05 13:00:00,2150-05-13 15:08:00
54218,191880,12970,0,2128-10-25 14:32:00,2128-11-08 18:00:00
43660,174004,19119,0,2139-01-04 08:00:00,2139-01-11 16:20:00
55629,194289,20208,0,2132-08-14 07:15:00,2132-08-28 16:07:00
23173,139265,47591,0,2169-01-26 07:06:00,2169-02-09 12:24:00


0    0.806328
1    0.193672
Name: hcc_cd_135, dtype: float64

**Demographic Features**
---
Read in the patients table and create demographic features.

In [3]:
patients = read_prod_data('patients')
demographic_features = create_demographics_features(admissions, patients)
del patients
printer('demographic features')
demographic_features.shape
demographic_features.columns
demographic_features.sample(5)

patients (46520, 8)
demographic features
-------------------


(58976, 48)

Index(['hadm_id', 'ft_age', 'ft_gender', 'ft_admit_type_elective',
       'ft_admit_type_emergency', 'ft_admit_type_newborn',
       'ft_admit_type_urgent', 'ft_race_american_indian_alaska_native',
       'ft_race_american_indian_alaska_native_federally_recognized_tribe',
       'ft_race_asian', 'ft_race_asian_asian_indian',
       'ft_race_asian_cambodian', 'ft_race_asian_chinese',
       'ft_race_asian_filipino', 'ft_race_asian_japanese',
       'ft_race_asian_korean', 'ft_race_asian_other', 'ft_race_asian_thai',
       'ft_race_asian_vietnamese', 'ft_race_black_african',
       'ft_race_black_african_american', 'ft_race_black_cape_verdean',
       'ft_race_black_haitian', 'ft_race_caribbean_island',
       'ft_race_hispanic_latino_central_american_(other)',
       'ft_race_hispanic_latino_colombian', 'ft_race_hispanic_latino_cuban',
       'ft_race_hispanic_latino_dominican',
       'ft_race_hispanic_latino_guatemalan',
       'ft_race_hispanic_latino_honduran', 'ft_race_hispanic_la

,hadm_id,ft_age,ft_gender,ft_admit_type_elective,ft_admit_type_emergency,ft_admit_type_newborn,ft_admit_type_urgent,ft_race_american_indian_alaska_native,ft_race_american_indian_alaska_native_federally_recognized_tribe,ft_race_asian,ft_race_asian_asian_indian,ft_race_asian_cambodian,ft_race_asian_chinese,ft_race_asian_filipino,ft_race_asian_japanese,ft_race_asian_korean,ft_race_asian_other,ft_race_asian_thai,ft_race_asian_vietnamese,ft_race_black_african,ft_race_black_african_american,ft_race_black_cape_verdean,ft_race_black_haitian,ft_race_caribbean_island,ft_race_hispanic_latino_central_american_(other),ft_race_hispanic_latino_colombian,ft_race_hispanic_latino_cuban,ft_race_hispanic_latino_dominican,ft_race_hispanic_latino_guatemalan,ft_race_hispanic_latino_honduran,ft_race_hispanic_latino_mexican,ft_race_hispanic_latino_puerto_rican,ft_race_hispanic_latino_salvadoran,ft_race_hispanic_or_latino,ft_race_middle_eastern,ft_race_multi_race_ethnicity,ft_race_native_hawaiian_or_other_pacific_islander,ft_race_other,ft_race_patient_declined_to_answer,ft_race_portuguese,ft_race_south_american,ft_race_unable_to_obtain,ft_race_unknown_not_specified,ft_race_white,ft_race_white_brazilian,ft_race_white_eastern_european,ft_race_white_other_european,ft_race_white_russian
5500,109297,42.321697,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
54520,192384,56.169747,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
56369,195532,81.284052,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
51623,187511,48.281999,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
10736,118114,57.393566,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


**HCC's for Comorbidities**
---
Create features for HCCs that are comorbidities.

In [4]:
diabetes_hcc_feature = create_hcc_feature(hcc_data, 
                                          label='_19', 
                                          rename_as='hcc_cd_19_dbtes_wo_comp')
ckd5_hcc_feature = create_hcc_feature(hcc_data, 
                                      label='_136', 
                                      rename_as='hcc_cd_136_ckd_stg_5')
ckd4_hcc_feature = create_hcc_feature(hcc_data, 
                                      label='_137', 
                                      rename_as='hcc_cd_137_ckd_stg_4')
chf_hcc_feature = create_hcc_feature(hcc_data, 
                                     label='_85',
                                     rename_as='hcc_cd_85_chf')
vascular_disease_hcc_feature = create_hcc_feature(hcc_data, 
                                                  label='_108', 
                                                  rename_as='hcc_cd_108_vascular')
del hcc_data
printer('hcc features')
diabetes_hcc_feature.shape
ckd5_hcc_feature.shape
ckd4_hcc_feature.shape
chf_hcc_feature.shape
vascular_disease_hcc_feature.shape
vascular_disease_hcc_feature.sample(5)

hcc features
-------------------


(58976, 2)

(58976, 2)

(58976, 2)

(58976, 2)

(58976, 2)

,hadm_id,hcc_cd_108_vascular
47181,179842,0
37324,163366,0
14740,124948,0
4517,107648,0
45886,177735,0


In [5]:
# admissions, icu stays
icustays = read_prod_data('icustays')
prior_admission_features = create_prior_admissions(admissions, icustays)
del icustays

icustays (61532, 12)


In [ ]:
# prescription
prescriptions = read_prod_data('prescriptions')
nephrotoxin_features = add_nephrotoxin_features(prescriptions, admissions)
del prescriptions

In [ ]:
# cptevents
cptevents = read_prod_data('cptevents')
contrast_imaging_feature = create_contrast_imaging_feature(cptevents)
del cptevents

**Charts Data**
---
View the feature space that's created for the chart data. Each feature space looks at another data point in the charts table.

In [6]:
one_bin = True

In [ ]:
d_items = read_prod_data('d_items')

if not one_bin:
    bins = [hex(i)[2] + c for i in range(0, 16) for c in [hex(d)[2] for d in range(0, 16)]]   
else:
    bins = ['0']

# chart features
chart_features = [charts_data_wrapper(b, d_items, df, demographic_features) for b in bins]
chart_features = pd.concat(chart_features, sort=False)

del d_items
printer('charts features')
chart_features.shape
chart_features.columns
chart_features.sample(5)

d_items (12487, 10)
chartevents (20756890, 15)
merge statistics
creatinine features
-------------------
urine features
-------------------


In [ ]:





    



    # chart data



    data.to_csv(result_dir + 'features.csv', index=False)


**Merge & Create Final Dataframe**
---

In [ ]:
features = [
    df.drop(['subject_id', 'admittime', 'dischtime'], axis=1),
    demographic_features,
    chart_features_by_bin,
    diabetes_hcc_feature,
    ckd4_hcc_feature,
    ckd5_hcc_feature,
    chf_hcc_feature,
    vascular_disease_hcc_feature,
    prior_admission_features,
    nephrotoxin_features,
    contrast_imaging_feature
]

data = merge_features(features)
printer('final dataframe')
print(data.shape)